# Ejercicio (resuelto) - Reconocimiento de texto

En esta sesión implementaremos un reconocedor de texto (*Optical Character Recognition*, OCR) mediante un enfoque encoder-decoder. La arquitectura a implementar seguirá una estructura como la descrita en la siguiente imágen:

![Arquitectura](https://drive.google.com/uc?id=1nti512wD3j5xUuzlHh8GdOfxs7jq2bf7)


### Código Keras paso a paso

En primer lugar vamos a implementar una función que nos genere una imagen con el texto que se le pasa por parámetro. Para ello utilizaremos la librería OpenCV.

In [ ]:
import cv2
import numpy as np

# Función que genera una imagen con el texto pasado por parámetro
def generateText(text):
    # Caracteristicas del texto a generar
    font = cv2.FONT_HERSHEY_SCRIPT_SIMPLEX # Fuente que simula escritura manuscrita
    font_scale = 4
    margin = 100
    thickness = 4
    color = 255

    # Calculamos lo que ocuparía el texto con las características anteriores
    size = cv2.getTextSize(text, font, font_scale, thickness)
    text_width = size[0][0]
    text_height = size[0][1]

    # Creamos una imagen vacía
    image = np.zeros((text_height+margin,text_width+margin*2),'uint8')

    # Calculamos la posición donde poner el texto
    x = margin
    y = (image.shape[0] + text_height) // 2

    # Pintamos el texto en la imagen
    cv2.putText(image, text, (x, y), font, font_scale, color, thickness)

    # Devolvemos la imagen
    return image

Podemos comprobar qué aspecto tienen las imagenes generadas.

In [ ]:
import matplotlib
from matplotlib import pyplot
%matplotlib inline
matplotlib.rcParams['figure.dpi']= 150


# Generamos una imagen con el texto `Deep Learning`
texto_img = generateText("Deep Learning")

# Mostramos la imagen con matplotlib
pyplot.imshow(texto_img)

Con esta función podemos generar las imágenes pero nos falta saber qué vocabulario vamos a considerar. En lugar de generar palabras aleatorias, vamos a utilizar conjuntos de palabras pre-establecidos. En concreto, vamos a crear *nombres propios* a partir de palabras en mayúsculas.

In [ ]:
import urllib.request

# Descargamos un vocabulario de la web (como en la Sesion 3.1.1)
word_url = "http://svnweb.freebsd.org/csrg/share/dict/words?view=co&content-type=text/plain"
words = urllib.request.urlopen(word_url).read().decode().splitlines()

# Filtramos las palabras que solo tengan minusculas
words  = [word for word in words if word.islower()]

# Comprobación
print('Num. palabras: {}'.format(len(words)))
print('Palabra (0): {}'.format(words[0]))
print('Palabra (100): {}'.format(words[100]))
print('Palabra (1000): {}'.format(words[1000]))
print('Palabra (10000): {}'.format(words[10000]))

Nuestro conjunto de entrenamiento va a estar compuesto por muestras que contienen dos palabras de este conjunto (como si fueran un nombre y un apellido). Seleccionaremos al azar 5000 combinaciones para obtener nuestro conjunto de entrenamiento.

In [ ]:
import random

# Limite del conjunto
training_set_size = 5000

# Par de entrenamiento
X = []
Y = []

# Escogemos `training_set_size` palabras al azar
for n in range(training_set_size):
    Y.append(random.choice(words))

# Creamos las correspondientes imagenes
for name in Y:
    X.append(generateText(name))

# Comprobación
print('Tamaño del conjunto de entrenamiento (X): {}'.format(len(X)))
print('Tamaño del conjunto de entrenamiento (Y): {}'.format(len(Y)))

# Ejemplo de muestra
pyplot.imshow(X[0])                         # Pintar por pantalla la imagen (X)
print('Ejemplo de Y (0): {}'.format(Y[0]))  # Imprimir la palabra en texto (Y)

Los datos vienen ordenados alfabéticamente: la red podría utilizar el orden para *ahorrarse* información en el vector de contexto, lo cual podría ser perjudicial para el aprendizaje. Así pues, barajamos los datos manteniendo el alineamiento con el *ground-truth*

In [ ]:
import random

# Enlazamos ambas listas
common_set = list(zip(X, Y))

# Se barajan y se desenlazan de nuevo
random.shuffle(common_set)
X[:], Y[:] = zip(*common_set)

# Comprobación
print('Tamaño del conjunto de entrenamiento (X): {}'.format(len(X)))
print('Tamaño del conjunto de entrenamiento (Y): {}'.format(len(Y)))

# Ejemplo de muestra
pyplot.imshow(X[0])                         # Pintar por pantalla la imagen (X)
print('Ejemplo de Y (0): {}'.format(Y[0]))  # Imprimir la palabra en texto (Y)


Vamos ahora a crear el modelo de red neuronal. Antes, necesitamos establecer las condiciones de las imágenes de entrada, ya que la red convolucional necesita unas dimensiones pre-establecidas para ser definida (¿seguro?).

Con el objetivo de no distorsionar la letra, vamos a establecer un número de píxeles de altura fijos, y el ancho se hará acorde manteniendo la proporción.

In [ ]:
# Función que rescala a una altura fija y mantiene la proporción ancho/alto
def resize(image, height):
    width = int(float(height * image.shape[1]) / image.shape[0])
    sample_img = cv2.resize(image, (width, height))
    return sample_img

# Establecemos la altura a 40 píxeles
img_height = 40

# Realizamos el rescalado para todas las imagenes
for idx,image in enumerate(X):
  X[idx] = resize(image,img_height)

# Comprobación
pyplot.imshow(X[0])

Así pues, la red convolucional recibirá imagenes del alto establecido y el ancho máximo de entre todas las imágenes. Por lo tanto, necesitamos hacer *padding*: aquellas imagenes con un ancho real menor a este valor se rellenarán con valores nulos 0 en este caso.

In [ ]:
# Calculamos la anchura máxima
max_image_len = max([image.shape[1] for image in X])

# Creamos un único paquete de datos
encoder_input = np.zeros((len(X),img_height,max_image_len), dtype=np.float)

# Encajamos la imagen en su posición correspondiente
for idx, image in enumerate(X):
    encoder_input[idx][:,:image.shape[1]] = image

# Comprobación
pyplot.imshow(encoder_input[0])

El paquete de imágenes todavía no esta preparado para ser usado por Keras, ya que ahora mismo cada imagen solo tiene alto y ancho. Así pues, necesitamos asignarle un número de canales, en este caso 1 al ser en escala de grises. Para ello, basta con expandir el número de dimensiones, sin modificar en absoluto los valores de las matrices.

Además, sobre estas imagenes, vamos a utilizar un pequeño truco que ayuda a la convergencia del proceso de aprendizaje: las imágenes, en lugar de mostrar valores entre 0 y 255 (niveles de gris), van a estar normalizadas entre 0 y 1.

In [ ]:
# Normalizamos los valores de los píxeles
encoder_input = encoder_input / 255.

# Expandimos la última dimension (alto,ancho) -> (alto,ancho,1)
encoder_input = np.expand_dims(encoder_input, axis=-1)

# Comprobación
print(encoder_input.shape)


Ahora es necesario preparar los datos de salida. En primer lugar, vamos a incluir los caracteres de inicio y final de frase, necesarios para el decoder.


In [ ]:
# Caracteres de comienzo y final
output_sos = '<'
output_eos = '>'

# Se añaden a todos los nombres
Y = [output_sos + name + output_eos for name in Y]

# Comprobación
print(Y[0])

Para la parte del decoder (las letras reconocidas), también necesitamos fijar la longitud máxima de las secuencias, ya que vamos a realizar un único paquete de datos. Como ya hemos comentado, las redes recurrentes tienen la ventaja de adaptarse a anchos variables; sin embargo, los modelos se entrenan usando tensores, que deben tener unas dimensiones fijas.

In [ ]:
# Conjuntos de caracteres y conversores
alphabet = set()

for name in Y:
    alphabet.update(list(name))

alphabet_len = len(alphabet)

print('Hay un total de ' + str(alphabet_len) + ' caracteres: ' + str(alphabet))

alphabet_from_char_to_int = dict([(char, i) for i, char in enumerate(alphabet)])
alphabet_from_int_to_char = dict([(i, char) for i, char in enumerate(alphabet)])



Creamos los paquetes de entrada y salida del decoder:

In [ ]:
max_output_len = max([len(name) for name in Y])

decoder_input = np.zeros((len(Y),max_output_len,alphabet_len), dtype=np.float)
decoder_output = np.zeros((len(Y),max_output_len,alphabet_len), dtype=np.float)

for idx_s, output_sentence in enumerate(Y):
    for idx_c, char in enumerate(output_sentence):
        decoder_input[idx_s][idx_c][alphabet_from_char_to_int[char]] = 1.
        if idx_c > 0:
            decoder_output[idx_s][idx_c-1][alphabet_from_char_to_int[char]] = 1.


Ya tenemos nuestro paquete de datos formado por tres vectores (entrada del encoder, entrada del decoder y salida del decoder).

In [ ]:
print ('Encoder input: ' + str(encoder_input.shape))
print ('Decoder input: ' + str(decoder_input.shape))
print ('Decoder output: ' + str(decoder_output.shape))

Definamos ahora una función que recibe los parámetros del problema y devuelve un modelo encoder (CNN) - decoder (RNN) ajustado a dichos parámetros.

In [ ]:
from keras.layers import Input, Dense, Flatten
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.layers.recurrent import LSTM
from keras.models import Model
from keras import backend as K

def getmodel(img_height, img_width, alphabet_len):

  # Comprobamos el formato de datos esperado
  input_data = Input(name='input', shape=(img_height, img_width, 1))


  # Encoder como CNN de tres capas
  inner = Conv2D(32, (3, 3), padding='same', activation='relu')(input_data)
  inner = MaxPooling2D(pool_size=(2,2))(inner)

  inner = Conv2D(64, (3, 3), padding='same', activation='relu')(inner)
  inner = MaxPooling2D(pool_size=(2,2))(inner)

  inner = Conv2D(128, (3, 3), padding='same', activation='relu')(inner)
  inner = MaxPooling2D(pool_size=(2,2))(inner)

  # Conexion CNN con LSTM
  x = Flatten()(inner)
  h_0 = Dense(256, activation='tanh')(x)
  c_0 = Dense(256, activation='tanh')(x)

  initial_state = [h_0, c_0]

  # Decoder como RNN
  decoder_inputs = Input(shape=(None, alphabet_len))
  decoder_outputs, _, _ = LSTM(256, return_sequences=True, return_state=True)(decoder_inputs, initial_state=initial_state)
  decoder_dense = Dense(alphabet_len, activation='softmax')
  decoder_outputs = decoder_dense(decoder_outputs)

  # Creamos el modelo único
  model = Model([input_data, decoder_inputs], decoder_outputs)
  model.compile(optimizer='rmsprop', loss='categorical_crossentropy')
  model.summary()

  return model

# Extraemos los parámetros de los paquetes creados
model = getmodel(encoder_input.shape[1],encoder_input.shape[2],alphabet_len)

El modelo está listo para ser entrenado y evaluado. Antes, vamos a realizar una partición de los datos disponibles para entrenamiento y validación: 99% y 1% respectivamente.

In [ ]:
val_split = 0.01
idx_split = int(len(X)*val_split)

# Conjunto de entrenamiento: paquetes codificados
x_train = encoder_input[idx_split:]
y_train = decoder_input[idx_split:]
t_train = decoder_output[idx_split:]

# Conjunto de validación: paquetes codificados + salida esperada
x_val = encoder_input[:idx_split]
y_val = decoder_input[:idx_split]
t_val = decoder_output[:idx_split]
i_val = Y[:idx_split]

Entrenamos durante 15 épocas, monitorizando cuantitativamente por época (loss) y cualitativamente cada 3 épocas.

In [ ]:
# Entrenamos durante 5 super-epocas
for super_epoch in range(5):
    print('Super-epoca: ' + str(super_epoch))

    # 3 épocas en cada iteración
    model.fit([x_train, y_train], t_train, verbose=1, batch_size=16, epochs=3, validation_data=[[x_val,y_val], t_val])

    # Predecimos sobre el conjunto de validación
    batch_prediction = model.predict([x_val,y_val],batch_size=16)

    # Comprobamos las 5 primeras imágenes
    for idx,sentence_prediction in enumerate(batch_prediction[:5]):

        raw_predicted_sequence = [alphabet_from_int_to_char[char] for char in np.argmax(sentence_prediction,axis=1)]

        predicted_sentence = output_sos
        for char in raw_predicted_sequence:
            predicted_sentence += char
            if char == output_eos:
                break

        print( 'Predicción\t: ' + str(predicted_sentence) )
        print( 'Sal. esperada\t: ' + str(i_val[idx]) )
        print()


Muy lentamente, el modelo va aprendiendo a reconocer los caracteres de las imágenes.

### Consideraciones

Los resultados obtenidos en este proyecto seguramente están lejos de la fiabilidad que alcanzan los sistemas actuales. No obstante, la formulación de los modelos del estado del arte son similares: la diferencia rádica especialmente en el tamaño del modelo de red y la cantidad de datos para entrenar. En este ejemplo es sencillo aumentar tanto el tamaño de la red como el tamaño del conjunto de entrenamiento.

También existen otras cuestiones que quizá hayas obviado en este ejercicio como la búsqueda de hiper-parámetros adecuados, funciones generadoras para reducir el *padding* o modelos de atención (ver final de Seccion 3.2.1).